<a href="https://colab.research.google.com/github/Keidhnn/ML-A11.2023.15322-UAS1/blob/main/ML_A11_2023_15322_UAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "authorship_tag": "ABX9TyMEVZbneu03u9CZd8Ewk3Sr"
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "intro"
      },
      "source": [
        "# Prediksi dan Klasifikasi Fauna Menggunakan Machine Learning Sederhana\n",
        "\n",
        "Nama: Peter V\n",
        "NIM: A11.2023.15322\n",
        "Kelompok: A11.4406 "
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "summary"
      },
      "source": [
        "## Ringkasan Proyek\n",
        "Proyek ini bertujuan untuk mendemonstrasikan aplikasi machine learning dalam pemantauan dan konservasi fauna. Kami akan membangun dua jenis model:\n",
        "1.  Model Regresi Linear Sederhana: Untuk memprediksi jumlah individu fauna di suatu area berdasarkan fitur lingkungan (menggunakan data sintetis).\n",
        "2.  Model Klasifikasi Gambar (Deep Learning): Untuk mengidentifikasi jenis fauna dari citra, mensimulasikan integrasi dengan model yang sudah ada atau memuatnya jika tersedia.\n",
        "\n",
        "Integrasi kedua model ini dapat membantu dalam perencanaan survei populasi (prediksi jumlah) dan verifikasi identifikasi spesies di lapangan (klasifikasi gambar), yang krusial untuk upaya konservasi dan pemantauan ekosistem."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "problem"
      },
      "source": [
        "## Permasalahan\n",
        "-   Bagaimana cara memprediksi jumlah individu fauna di suatu area tertentu dengan efisien, berdasarkan data fitur lingkungan yang tersedia?\n",
        "-   Bagaimana mengidentifikasi jenis fauna secara otomatis dari citra yang diambil di lapangan, terutama untuk spesies yang terancam punah?"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "objective"
      },
      "source": [
        "## Tujuan\n",
        "-   Mengimplementasikan dan mengevaluasi model regresi linear sederhana untuk memprediksi jumlah fauna.\n",
        "-   Menyajikan konsep penggunaan model klasifikasi gambar (yang sudah terlatih) untuk identifikasi spesies fauna.\n",
        "-   Menjelaskan potensi integrasi kedua pendekatan machine learning ini dalam konteks konservasi."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "dataset_regression"
      },
      "source": [
        "## Penjelasan Dataset (Data Sintetis untuk Regresi)\n",
        "Untuk demonstrasi regresi linear, kami akan menggunakan dataset sintetis yang sederhana, mirip dengan studi kasus 'nilai siswa' namun diinterpretasikan ulang dalam konteks fauna:\n",
        "-   Fitur (Variabel Independen/Prediktor): Lingkungan\n",
        "    -   Merepresentasikan faktor-faktor lingkungan yang mempengaruhi populasi fauna, seperti kualitas habitat, ketersediaan sumber daya, atau tingkat polusi. Nilai yang lebih tinggi dapat diinterpretasikan sebagai kondisi lingkungan yang lebih baik (atau buruk, tergantung konteks).\n",
        "-   Label (Variabel Dependen/Respons/Outcome): Jumlah_Fauna\n",
        "    -   Merepresentasikan estimasi jumlah individu fauna yang dapat ditemukan di area dengan kondisi lingkungan tertentu. Kami berharap ada hubungan positif antara Lingkungan dan Jumlah_Fauna."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "dataset_classification"
      },
      "source": [
        "## Penjelasan Dataset (Data Gambar Fauna untuk Klasifikasi)\n",
        "Untuk demonstrasi klasifikasi gambar, kami akan mengacu pada dataset 'Indonesian Endangered Animal Dataset', yang dapat digunakan untuk mengidentifikasi spesies hewan tertentu.\n",
        "-   Fitur (Input): Citra Hewan\n",
        "    -   Berisi koleksi gambar dari berbagai spesies fauna yang terancam punah di Indonesia.\n",
        "-   Label (Output): Kategori Hewan\n",
        "    -   Setiap gambar dikategorikan ke dalam salah satu dari 6 kelas spesies: 'Bekantan', 'Beruang Madu', 'Burung', 'Gajah Asia', 'Macan Tutul', dan 'Tapir'."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "libraries"
      },
      "outputs": [],
      "source": [
        "# Import library yang dibutuhkan\n",
        "import numpy as np # Untuk operasi numerik\n",
        "import pandas as pd # Untuk manipulasi data (tidak digunakan secara langsung untuk data sintetis ini, tapi baik untuk data tabular)\n",
        "import matplotlib.pyplot as plt # Untuk visualisasi data\n",
        "from sklearn.linear_model import LinearRegression # Model regresi linear\n",
        "import os # Untuk berinteraksi dengan sistem operasi (misalnya, path file)\n",
        "from PIL import Image # Untuk memproses gambar\n",
        "import tensorflow as tf # Kerangka kerja machine learning untuk deep learning\n",
        "from tensorflow.keras.preprocessing.image import load_img, img_to_array # Untuk memuat dan memproses gambar untuk model\n",
        "from tensorflow.keras.models import load_model # Untuk memuat model Keras yang sudah disimpan\n",
        "import ipywidgets as widgets # Untuk membuat elemen UI interaktif di Jupyter\n",
        "from IPython.display import display # Untuk menampilkan widget di Jupyter\n",
        "\n",
        "# (Opsional) Install pustaka tambahan jika belum ada (biasanya sudah ada di Colab)\n",
        "# !pip install numpy matplotlib\n",
        "# !pip install tensorflow"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "data_process_regression"
      },
      "source": [
        "## Proses Data (Regresi Linear Sederhana)\n",
        "Kita akan membuat data Lingkungan dan Jumlah_Fauna secara sintetis, lalu memvisualisasikannya."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "generate_data_regression"
      },
      "outputs": [],
      "source": [
        "# x: variabel lingkungan (misal, tingkat kualitas habitat)\n",
        "# y: jumlah individu fauna (misal, jumlah spesies yang ditemukan)\n",
        "n_samples = 6 # Jumlah sampel data\n",
        "x_environment = np.array([1, 2, 3, 4, 5, 6]) # Nilai variabel lingkungan\n",
        "y_fauna_count = np.array([70, 80, 75, 85, 81, 90]) # Jumlah fauna yang teramati"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "focused_cell_id": "YORgGo4Teczy",
          "order": [
            "jdn8nGvwde_a",
            "Tf82Y5jXeI9S",
            "SC11Gy_WeTyy",
            "gsk-IS8EeY_9",
            "YORgGo4Teczy",
            "ifr9XhlgepbI",
            "8gY7buj3e_oc",
            "Do3eMVoFfHiT",
            "z9xy6y19fLXh",
            "zLgcK8csfYu6"
          ]
        },
        "id": "scatter_plot_regression"
      },
      "outputs": [],
      "source": [
        "# Scatter plot untuk x_environment dan y_fauna_count\n",
        "plt.scatter(x_environment, y_fauna_count)\n",
        "plt.title('Gambar Scatter Plot: Lingkungan vs. Jumlah Fauna')\n",
        "plt.xlabel('Variabel Lingkungan (Independent/Prediktor)')\n",
        "plt.ylabel('Jumlah Fauna (Dependent/Respons/Outcome)')\n",
        "plt.grid(True)\n",
        "plt.show() # Menampilkan plot"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "model_regression"
      },
      "source": [
        "## Model Regresi Linear Sederhana\n",
        "Regresi Linear Sederhana adalah metode statistik untuk memodelkan hubungan linear antara variabel bebas (Lingkungan) dan variabel terikat (Jumlah_Fauna)."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "train_model_regression"
      },
      "outputs": [],
      "source": [
        "# Mengubah bentuk array x agar sesuai dengan input sklearn (2D array)\n",
        "X_environment_reshaped = x_environment.reshape(-1, 1)\n",
        "\n",
        "# Membuat instance model regresi linear\n",
        "linear_model = LinearRegression()\n",
        "\n",
        "# Melatih model menggunakan data yang ada\n",
        "linear_model.fit(X_environment_reshaped, y_fauna_count)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "plot_regression_line"
      },
      "outputs": [],
      "source": [
        "# Scatter plot data asli\n",
        "plt.scatter(x_environment, y_fauna_count)\n",
        "\n",
        "# Plot garis regresi (prediksi dari model)\n",
        "plt.plot(x_environment, linear_model.predict(X_environment_reshaped), color='red') # Memplot garis prediksi\n",
        "\n",
        "plt.title('Gambar Scatter Plot dengan Garis Regresi')\n",
        "plt.xlabel('Variabel Lingkungan (Independent/Prediktor)')\n",
        "plt.ylabel('Jumlah Fauna (Dependent/Respons/Outcome)')\n",
        "plt.grid(True)\n",
        "plt.show() # Menampilkan plot"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "prediction_regression"
      },
      "source": [
        "## Prediksi dengan Model Regresi\n",
        "Kita bisa menggunakan model yang sudah dilatih untuk memprediksi jumlah fauna berdasarkan nilai variabel lingkungan baru."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "new_prediction"
      },
      "outputs": [],
      "source": [
        "# Contoh prediksi untuk nilai lingkungan baru (misal, tingkat kualitas habitat 7)\n",
        "new_environment_value = np.array([[7]])\n",
        "predicted_fauna_count = linear_model.predict(new_environment_value)\n",
        "\n",
        "print(f\"Jika variabel lingkungan = {new_environment_value[0][0]}, maka prediksi jumlah fauna adalah sekitar {predicted_fauna_count[0]:.2f}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "integration_classification"
      },
      "source": [
        "## Integrasi dengan Model Klasifikasi Fauna (Deep Learning)\n",
        "Setelah memprediksi potensi jumlah fauna di suatu area, langkah selanjutnya adalah mengidentifikasi jenis fauna yang ada. Ini dapat dilakukan menggunakan model klasifikasi gambar (seperti yang dilatih dalam fauna classification.ipynb).\n",
        "\n",
        "Catatan Penting: Untuk bagian ini, kami mensimulasikan penggunaan model klasifikasi yang sudah terlatih. Jika Anda memiliki file model Keras (.keras) yang telah dihasilkan dari pelatihan sebelumnya (misalnya, dari fauna classification.ipynb setelah sel pelatihan dijalankan dan model disimpan), Anda dapat menempatkannya di direktori yang benar dan model akan dimuat untuk inferensi.\n",
        "\n",
        "Model klasifikasi ini dilatih untuk mengenali 6 kelas fauna yang terancam punah di Indonesia: 'Bekantan', 'Beruang Madu', 'Burung', 'Gajah Asia', 'Macan Tutul', dan 'Tapir'."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "download_and_extract_dataset_classification"
      },
      "outputs": [],
      "source": [
        "# Langkah-langkah untuk mengunduh dan mengekstrak dataset fauna\n",
        "# (Hanya perlu dijalankan jika dataset belum ada dan Anda ingin menggunakannya untuk melatih/mengecek model)\n",
        "\n",
        "# Pastikan file kaggle.json sudah ada di direktori /content/\n",
        "# !mkdir -p ~/.kaggle\n",
        "# !cp kaggle.json ~/.kaggle/\n",
        "# !chmod 600 ~/.kaggle/kaggle.json\n",
        "\n",
        "# Unduh dataset\n",
        "# !kaggle datasets download -d nadyanurfadhila/indonesian-endangered-animal\n",
        "\n",
        "# Ekstrak dataset\n",
        "# !unzip indonesian-endangered-animal.zip -d ./indonesian_endangered_animal_dataset\n",
        "\n",
        "# Verifikasi gambar (opsional, jika Anda mengekstrak dataset)\n",
        "# base_dir_fauna = './indonesian_endangered_animal_dataset/Cleaned Endangered Animal'\n",
        "# if not os.path.exists(base_dir_fauna):\n",
        "#     print(f\"Error: Directory '{base_dir_fauna}' not found. Please check the path.\")\n",
        "# else:\n",
        "#     def verify_images(directory):\n",
        "#         for folder_name in os.listdir(directory):\n",
        "#             folder_path = os.path.join(directory, folder_name)\n",
        "#             if os.path.isdir(folder_path):\n",
        "#                 for filename in os.listdir(folder_path):\n",
        "#                     file_path = os.path.join(folder_path, filename)\n",
        "#                     try:\n",
        "#                         img = Image.open(file_path)\n",
        "#                         img.verify()\n",
        "#                     except (IOError, SyntaxError) as e:\n",
        "#                         print(f'Found corrupted image: {file_path}')\n",
        "#                         os.remove(file_path)\n",
        "#     verify_images(base_dir_fauna)\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "load_and_use_classification_model"
      },
      "outputs": [],
      "source": [
        "# Asumsi: model klasifikasi terbaik sudah disimpan sebagai 'best_model.keras'\n",
        "# Jika belum ada, sel ini akan error. Anda bisa melatih model di fauna classification.ipynb dan menyimpannya.\n",
        "\n",
        "model_path = 'best_model.keras'\n",
        "\n",
        "if os.path.exists(model_path):\n",
        "    classification_model = load_model(model_path)\n",
        "    print(f\"Model klasifikasi '{model_path}' berhasil dimuat.\")\n",
        "\n",
        "    # Kelas sesuai urutan pelatihan model\n",
        "    class_labels_fauna = ['Bekantan', 'Beruang Madu', 'Burung', 'Gajah Asia', 'Macan Tutul', 'Tapir']\n",
        "\n",
        "    def predict_fauna_image(image_path_input):\n",
        "        try:\n",
        "            # Muat dan ubah ukuran gambar sesuai input model (128x128)\n",
        "            img = load_img(image_path_input, target_size=(128, 128))\n",
        "            # Konversi gambar menjadi array NumPy\n",
        "            img_array = img_to_array(img)\n",
        "            # Tambahkan dimensi batch (1, 128, 128, 3)\n",
        "            img_array = np.expand_dims(img_array, axis=0)\n",
        "            # Normalisasi jika model menggunakan gambar yang dinormalisasi (0-1)\n",
        "            img_array = img_array / 255.0\n",
        "\n",
        "            # Lakukan prediksi\n",
        "            predictions = classification_model.predict(img_array)\n",
        "            # Ambil indeks kelas dengan probabilitas tertinggi\n",
        "            predicted_class_index = np.argmax(predictions, axis=1)\n",
        "            # Ambil nama kelas yang diprediksi\n",
        "            predicted_label = class_labels_fauna[predicted_class_index[0]]\n",
        "            \n",
        "            return predicted_label\n",
        "        except Exception as e:\n",
        "            return f\"Error saat memproses gambar: {e}\"\n",
        "\n",
        "    # Widget untuk unggah gambar\n",
        "    upload_button = widgets.FileUpload(accept='image/*')\n",
        "    output_widget = widgets.Output()\n",
        "\n",
        "    def on_upload_button_clicked(change):\n",
        "        with output_widget:\n",
        "            output_widget.clear_output()\n",
        "            if upload_button.value:\n",
        "                # Ambil data gambar yang diunggah\n",
        "                uploaded_file_info = list(upload_button.value.values())[0]\n",
        "                image_content = uploaded_file_info['content']\n",
        "                uploaded_filename = uploaded_file_info['metadata']['name']\n",
        "                \n",
        "                # Simpan gambar yang diunggah sementara\n",
        "                temp_image_path = f'./{uploaded_filename}'\n",
        "                with open(temp_image_path, 'wb') as f:\n",
        "                    f.write(image_content)\n",
        "                \n",
        "                # Lakukan prediksi\n",
        "                fauna_prediction = predict_fauna_image(temp_image_path)\n",
        "                \n",
        "                print(f'Gambar {uploaded_filename} diklasifikasikan sebagai: {fauna_prediction}')\n",
        "                \n",
        "                # Hapus file sementara\n",
        "                os.remove(temp_image_path)\n",
        "            else:\n",
        "                print('Silakan unggah gambar untuk klasifikasi fauna.')\n",
        "\n",
        "    upload_button.observe(on_upload_button_clicked, names=['value'])\n",
        "\n",
        "    print(\"### Klasifikasi Fauna dari Gambar ###\")\n",
        "    print(\"Unggah gambar fauna untuk melihat klasifikasi:\")\n",
        "    display(upload_button, output_widget)\n",
        "\n",
        "else:\n",
        "    print(f\"Model klasifikasi 'best_model.keras' tidak ditemukan. Harap pastikan model sudah dilatih dan disimpan.\")\n",
        "    print(\"Anda bisa melatihnya dengan menjalankan notebook 'fauna classification.ipynb' dan menyimpan modelnya.\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "conclusion"
      },
      "source": [
        "## Kesimpulan\n",
        "Proyek ini telah berhasil mendemonstrasikan dua aplikasi machine learning yang berbeda namun saling melengkapi dalam konteks pemantauan fauna:\n",
        "\n",
        "1.  Regresi Linear Sederhana: Model ini mampu memprediksi Jumlah_Fauna berdasarkan variabel Lingkungan. Meskipun menggunakan data sintetis, konsep ini menunjukkan bagaimana analisis regresi dapat digunakan untuk estimasi kuantitatif dalam perencanaan konservasi (misalnya, memperkirakan populasi berdasarkan kondisi habitat).\n",
        "2.  Klasifikasi Gambar Fauna: Melalui integrasi (simulasi) model deep learning, kita dapat mengidentifikasi jenis spesies fauna dari gambar. Ini sangat berguna untuk verifikasi lapangan dan pengumpulan data spesies secara otomatis.\n",
        "\n",
        "Potensi Integrasi: Hasil prediksi dari model regresi (misalnya, 'area ini diperkirakan memiliki jumlah fauna X') dapat memicu pengiriman tim lapangan untuk melakukan survei lebih lanjut. Tim kemudian dapat menggunakan aplikasi berbasis model klasifikasi gambar ini untuk mengidentifikasi spesies yang mereka temui secara real-time.\n",
        "\n",
        "Keterbatasan:\n",
        "-   Data Regresi: Model regresi saat ini sangat sederhana dan menggunakan data sintetis. Untuk aplikasi nyata, diperlukan data lingkungan dan populasi fauna yang kompleks dan riil.\n",
        "-   Model Klasifikasi: Kinerja model klasifikasi (akurasi, precision, recall) sangat bergantung pada kualitas dan kuantitas dataset pelatihan, serta arsitektur dan hyperparameter tuning yang optimal. Model yang disimulasikan di sini hanya mewakili fungsionalitasnya; kinerja sebenarnya akan bervariasi.\n",
        "\n",
        "Pengembangan Masa Depan:\n",
        "-   Menggunakan dataset regresi yang lebih kaya dengan banyak fitur lingkungan (multivariat).\n",
        "-   Meningkatkan kompleksitas model regresi (misalnya, menggunakan Random Forest atau XGBoost).\n",
        "-   Melatih dan menguji model klasifikasi secara menyeluruh dengan dataset yang lebih besar dan beragam.\n",
        "-   Membangun aplikasi terintegrasi yang memungkinkan pengguna memprediksi jumlah dan mengunggah gambar langsung dari perangkat lapangan."
      ]
    }
  ]
}

NameError: name 'null' is not defined